# Make Logbook of  HoloSpec Logbook files fz

- author : Sylvie Dagoret-Campagne
- date   : 31 octobre 2018
- goal.  : try to generate a logbook 


Exemple de lecture de fichier avec pyfits

   import pyfits
   
   fits = pyfits.open(fits_image_filename)
   
   print fits.info()

** 2018-06-19 Premiers essais, tests du montage d'injection devant le CCD
   RepÃ©rage du spot sur les images, tests avec Thorlabs GT50-03, puis
   l'hologramme en transmission.

   CC-IN2P3: /sps/lsst/DataBE/lpnws5203/data/frames/20180619/

** 2018-06-21: Tentatives d'Ã©tudier la focalisation avec la lampe HgAr (PenRay)

   CC-IN2P3: /sps/lsst/DataBE/lpnws5203/data/frames/20180621/

** 2018-06-28: Scan en z avec l'hologramme en transmission.

   CC-IN2P3: /sps/lsst/DataBE/lpnws5203/data/frames/20180628/

** 2018-07-17: Mesure de transmission du Thorlabs GT50-03

   CC-IN2P3: /sps/lsst/DataBE/lpnws5203/data/frames/20180717/
   CC-IN2P3: /sps/lsst/DataBE/lpnws5203/data/frames/20180718/
   CC-IN2P3: /sps/lsst/DataBE/lpnws5203/data/frames/20180720/

** Pause estivale


* Octobre 2018

  Logbook dÃ©taillÃ© : 
  http://supernovae.in2p3.fr/~llg/LSST/AuxTel-holograms/logbooks/logbook-2018-octobre.pdf

** 20180926: Test hologramme de phase HoloPhP (photopolymÃ¨re)

   CC-IN2P3: /sps/lsst/DataBE/lpnws5203/data/frames/20180926/

** 20180927: Test HoloPhAg

   Etude avec la HgAr (placÃ©e dans la petite sphÃ¨re)
   modif. du montage pour Ãªtre Ã  58 mm
   L'amÃ©lioration de la focalisation est enfin visible
   Comparatifs avec le Thorlabs GT50-03

   CC-IN2P3: /sps/lsst/DataBE/lpnws5203/data/frames/20180927/

** 20180928: Scan en longueur d'onde
   
   QTH + monchromateur : scans en longueur d'onde
   mesures de transmission (On/Off)

   CC-IN2P3: /sps/lsst/DataBE/lpnws5203/data/frames/20180928/

   Lampe QTH en fin de vie... RÃ©sultats peu fiables.

   

In [164]:
from astropy.io import fits

In [165]:
from os import listdir
from os.path import isfile, join,isdir
import os

In [166]:
import re

In [167]:
import pandas as pd
import numpy as np

# Config

In [168]:
DUMP_FLAG=False

In [169]:
logbook_rootfilename="Logbook_holospec_2018"

In [170]:
file_excel=logbook_rootfilename+".xlsx"
file_csv=logbook_rootfilename+".csv"

In [171]:
def GetDisperserAndSource(subdirname):
    disperser="?"
    source ="?"
    comment="?"
    
    if subdirname=="20180619":
        disperser="Thorlabs-GT50-03"
    elif subdirname=="20180621":
        disperser="Thorlabs-GT50-03"
        source="HgAr"
    elif subdirname=="20180628":
        disperser="Thorlabs-GT50-03"
        source="HgAr"
        comment="Scan en z avec l'hologramme en transmission"
    elif subdirname=="20180717":
        disperser="Thorlabs-GT50-03"
        source="HgAr"
    elif subdirname=="20180718":
        disperser="Thorlabs-GT50-03"
        source="HgAr"
    elif subdirname=="20180720":
        disperser="Thorlabs-GT50-03"
        source="HgAr"
    elif subdirname=="20180926":
        disperser="HoloPhP"
        source=""
    elif subdirname=="20180927":
        disperser="HoloPhAg"
        source="HgAr"
        comment="Etude avec la HgAr (placee dans la petite sphere) modif du montage pour etreva 58 mm L'amelioration de la focalisation est enfin visible Comparatifs avec le Thorlabs GT50-03"
    elif subdirname=="20180928":
        disperser="HoloPhAg"
        source="QTH + monchromateur"
        comment="Scans en longueur d'onde mesures de transmission (On/Off), Lampe QTH en fin de vie... Resultats peu fiables."
    else:
        print "Not found {}".format(subdirname)
        
    return source,disperser,comment
           

# Where to search

In [172]:
topdir='/sps/lsst/DataBE/lpnws5203/data/frames'

In [173]:
mypath=topdir

In [174]:
all_subdir= [f for f in listdir(mypath) if isdir(join(mypath, f)) ]

In [175]:
print all_subdir

['20171201', '20160128', '20180706', '20170616', '20150515', '20150820', '20170516', '20161114', '20161215', '20160903', '20180306', '20141215', '20150417', '20180216', '20150624', '20180126', '20160421', '20150529', '20171108', '20150526', '20180717', '20171130', '20161121', '20171030', '20160523', '20160624', '20160725', '20160826', '20160927', '20150212', '20170523', '20171109', '20180227', '20180704', '20160404', '20160606', '20160707', '20170724', '20180910', '20170123', '20160928', '20170301', '20161205', '20170602', '20150918', '20160607', '20160708', '20160530', '20170302', '20180624', '20171110', '20180601', '20161110', '20170803', '20170202', '20160411', '20160512', '20180201', '20160125', '20170626', '20180111', '20141218', '20161111', '20170804', '20170108', '20150716', '20170704', '20181028', '20161027', '20161128', '20181030', '20150713', '20150929', '20180725', '20151127', '20150618', '20151124', '20170905', '20171117', '20150923', '20171017', '20160721', '20150612', '20

In [176]:
subdir=all_subdir[0]

In [177]:
mypath=os.path.join(topdir,subdir)

In [178]:
only_image_files = [f for f in listdir(mypath) if isfile(join(mypath, f)) and re.search('[.]fz$',f) ]

In [179]:
print only_image_files

['10_CCD1_20171201151706.fz', '10_CCD1_20171201155834.fz', '10_CCD1_20171201160013.fz', '10_CCD1_20171201153158.fz', '10_CCD1_20171201154116.fz', '10_CCD1_20171201155357.fz', '10_CCD1_20171201143335.fz', '10_CCD1_20171201152401.fz', '10_CCD1_20171201145222.fz', '10_CCD1_20171201153134.fz', '10_CCD1_20171201144957.fz', '10_CCD1_20171201143851.fz', '10_CCD1_20171201151358.fz', '10_CCD1_20171201145159.fz', '10_CCD1_20171201153113.fz', '10_CCD1_20171201144812.fz', '10_CCD1_20171201155654.fz', '10_CCD1_20171201155614.fz', '10_CCD1_20171201154453.fz', '10_CCD1_20171201151336.fz', '10_CCD1_20171201155812.fz', '10_CCD1_20171201144717.fz', '10_CCD1_20171201143352.fz', '10_CCD1_20171201155926.fz', '10_CCD1_20171201154617.fz', '10_CCD1_20171201143623.fz', '10_CCD1_20171201154523.fz', '10_CCD1_20171201160108.fz', '10_CCD1_20171201143249.fz', '10_CCD1_20171201155432.fz', '10_CCD1_20171201154012.fz', '10_CCD1_20171201143312.fz', '10_CCD1_20171201143523.fz', '10_CCD1_20171201151517.fz', '10_CCD1_2017

In [180]:
if DUMP_FLAG:
    for fits_image_filename in only_image_files:
        print '********************************************************************************'
        fullfilename=os.path.join(mypath,fits_image_filename)
        with fits.open(fullfilename) as hdus:
            hdus.info()

In [181]:
if DUMP_FLAG:
    for fits_image_filename in only_image_files:
        print '********************************************************************************'
        with fits.open(fits_image_filename) as hdus:
            print hdus[0].header

# Define infos to save in the logbook

In [182]:
print all_subdir

['20171201', '20160128', '20180706', '20170616', '20150515', '20150820', '20170516', '20161114', '20161215', '20160903', '20180306', '20141215', '20150417', '20180216', '20150624', '20180126', '20160421', '20150529', '20171108', '20150526', '20180717', '20171130', '20161121', '20171030', '20160523', '20160624', '20160725', '20160826', '20160927', '20150212', '20170523', '20171109', '20180227', '20180704', '20160404', '20160606', '20160707', '20170724', '20180910', '20170123', '20160928', '20170301', '20161205', '20170602', '20150918', '20160607', '20160708', '20160530', '20170302', '20180624', '20171110', '20180601', '20161110', '20170803', '20170202', '20160411', '20160512', '20180201', '20160125', '20170626', '20180111', '20141218', '20161111', '20170804', '20170108', '20150716', '20170704', '20181028', '20161027', '20161128', '20181030', '20150713', '20150929', '20180725', '20151127', '20150618', '20151124', '20170905', '20171117', '20150923', '20171017', '20160721', '20150612', '20

In [183]:
all_subdir=np.array(all_subdir)

# Select directories corresponding to HOLOSPEC

In [184]:
all_indexes=np.where(all_subdir>'20180601')[0]

In [185]:
all_indexes

array([  2,  20,  33,  38,  49,  67,  70,  73,  86,  88, 100, 105, 109,
       122, 129, 138, 142, 148, 150, 166, 171, 176, 179, 187, 221, 222,
       234, 238, 242, 247, 255, 260, 267, 292, 294, 296, 312, 316, 323,
       330, 337, 339, 352, 361, 365, 372, 385, 399, 401])

In [186]:
all_subdir2=all_subdir[all_indexes]

In [187]:
all_subdir2

array(['20180706', '20180717', '20180704', '20180910', '20180624',
       '20181028', '20181030', '20180725', '20180712', '20180622',
       '20180903', '20180723', '20180927', '20181023', '20180718',
       '20180628', '20181008', '20180721', '20180705', '20180716',
       '20180626', '20181029', '20180907', '20181016', '20180928',
       '20180905', '20180709', '20180619', '20180621', '20181001',
       '20181025', '20180926', '20181012', '20180627', '20180604',
       '20180720', '20180715', '20180625', '20181005', '20180906',
       '20181031', '20180726', '20180917', '20180607', '20181026',
       '20180904', '20180618', '20181024', '20180719'], dtype='|S8')

In [188]:
all_subdir2.sort()

In [189]:
all_subdir=all_subdir2

# Extract Header Info

In [190]:
all_subdirs=[]
all_subdirnum=[]
all_filenum=[]
all_filename=[]
all_filetag=[]
all_dateobs=[]
all_datefile=[]
all_mjdobs=[]
all_expo=[]
all_seqnum=[]
all_testtype=[]
all_imgtype=[]
all_filter=[]
all_disperser=[]
all_source=[]
all_disperser=[]
all_comment=[]

In [191]:
# loop on all subdir
subdir_num=0
for subdir in all_subdir:
    subdir_num+=1
    #define path
    mypath=os.path.join(topdir,subdir)
    only_image_files = [f for f in listdir(mypath) if isfile(join(mypath, f)) and re.search('[.]fz$',f) ]
    # loop on image file
    file_num=0
    for thefilename in only_image_files:
        file_num+=1
        fullfilename=os.path.join(mypath,thefilename)
        with fits.open(fullfilename) as hdus:
            hd=hdus[0].header
            all_subdirnum.append(subdir_num)
            all_subdirs.append(subdir)
            all_filenum.append(file_num)
            all_filename.append(hd["FILENAME"])
            all_filetag.append(hd["FILETAG"])
            all_mjdobs.append(hd["MJD-OBS"])
            all_datefile.append(hd["DATE"])
            all_dateobs.append(hd["DATE-OBS"])
            all_expo.append(hd["EXPTIME"])
            all_seqnum.append(hd["SEQNUM"])
            all_testtype.append(hd["SEQNUM"])
            all_imgtype.append(hd["IMGTYPE"])
            
            source,disperser,comment=GetDisperserAndSource(subdir)
            
            all_source.append(source)
            all_disperser.append(disperser)
            all_comment.append(comment)
            
        

Not found 20180604
Not found 20180607
Not found 20180607
Not found 20180607
Not found 20180607
Not found 20180607
Not found 20180607
Not found 20180607
Not found 20180607
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180618
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180622
Not found 20180624
Not found 20180624
Not found 20180624
Not found 20180624
Not found 20180624
Not found 20180624
Not found 20180624
Not found 20

In [192]:
columnsTitles=['subdirnum',
     'subdir', 
     'filenum',          
     'filename', 
     'filetag',
     'fileseqnum',
     "datefile",
     "dateobs",
     'mjdobs',
     'exposure',
     'runtype',
     'imgtype',
     'source',
     'disperser',
     'comment']

In [193]:
d = {'subdirnum':all_subdirnum,
     'subdir':all_subdirs,
     'filenum':all_filenum,
     'filename': all_filename, 
     'filetag': all_filetag,
     'fileseqnum':all_seqnum,
     "datefile":all_datefile,
     "dateobs":all_dateobs,
     'mjdobs':all_mjdobs,
     'exposure':all_expo,
     'runtype':all_testtype,
     'imgtype':all_imgtype,
     'source':all_source,
     'disperser':all_disperser,
     'comment':all_comment}

In [194]:
#print d

In [195]:
df=pd.DataFrame(data=d)

In [196]:
df.reindex(columns=columnsTitles)

,subdirnum,subdir,filenum,filename,filetag,fileseqnum,datefile,dateobs,mjdobs,exposure,runtype,imgtype,source,disperser,comment
0,1,20180604,1,10_CCD0_20180604143713.fz,04143713,0,2018-06-04T12:37:19.073586,2018-06-04T12:37:12.538139,58273.525840,0.000,0,Bias,?,?,?
1,2,20180607,1,10_CCD0_20180607122355.fz,07122355,0,2018-06-07T10:24:01.053834,2018-06-07T10:23:54.657169,58276.433271,0.000,0,Bias,?,?,?
2,2,20180607,2,10_CCD0_20180607123757.fz,07123757,0,2018-06-07T10:38:03.528200,2018-06-07T10:37:57.142565,58276.443022,0.000,0,Bias,?,?,?
3,2,20180607,3,10_CCD0_20180607122606.fz,07122606,0,2018-06-07T10:26:12.454433,2018-06-07T10:26:06.068364,58276.434792,0.000,0,Bias,?,?,?
4,2,20180607,4,10_CCD0_20180607120451.fz,07120451,0,2018-06-07T10:04:56.827194,2018-06-07T10:04:50.232994,58276.420026,0.000,0,Bias,?,?,?
5,2,20180607,5,10_CCD0_20180607120550.fz,07120550,0,2018-06-07T10:05:56.234251,2018-06-07T10:05:49.758189,58276.420715,0.000,0,Bias,?,?,?
6,2,20180607,6,10_CCD0_20180607121032.fz,07121032,0,2018-06-07T10:10:37.943378,2018-06-07T10:10:31.540060,58276.423976,0.000,0,Bias,?,?,?
7,2,20180607,7,10_CCD0_20180607123859.fz,07123859,0,2018-06-07T10:39:04.530849,2018-06-07T10:38:58.149567,58276.443729,0.000,0,Bias,?,?,?
8,2,20180607,8,10_CCD0_20180607122726.fz,07122726,0,2018-06-07T10:27:32.339426,2018-06-07T10:27:25.954047,58276.435717,0.000,0,Bias,?,?,?
9,3,20180618,1,10_CCD1_20180618180102.fz,18180102,0,2018-06-18T16:01:17.430475,2018-06-18T16:01:01.780372,58287.667382,5.000,0,Acquisition,?,?,?


In [197]:
cols = list(df.columns.values)

In [198]:
print cols

['comment', 'datefile', 'dateobs', 'disperser', 'exposure', 'filename', 'filenum', 'fileseqnum', 'filetag', 'imgtype', 'mjdobs', 'runtype', 'source', 'subdir', 'subdirnum']


# Save Files

In [202]:
df.to_csv(file_csv)

In [203]:
df.to_excel(file_excel)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 31: ordinal not in range(128)

In [201]:
writer = pd.ExcelWriter(file_excel)
df.to_excel(writer,'Sheet1')
writer.save()

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 31: ordinal not in range(128)